# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

from torch import cuda

from absa_models.ABSA_BERT_Dropout_CNN_BiLSTM_Linear import ABSA_BERT_Dropout_CNN_BiLSTM_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error()

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ABSA_SemEval16_Restaurants_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,absa_tags
0,Judging from previous posts this used to be a ...,"[Judging, from, previous, posts, this, used, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,"We, there were four of us, arrived at noon - t...","[We, ,, there, were, four, of, us, ,, arrived,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"They never brought us complimentary noodles, i...","[They, never, brought, us, complimentary, nood...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [10]:
SEQ_LEN = 512

In [11]:
MODEL_OUTPUT = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/models/bert_pre_trained_dropout_cnn_bilstm_linear.pth'
STATS_OUTPUT = '../../../results/ABSA/SemEval16 - Task 5 - Restaurants/stats/bert_pre_trained_dropout_cnn_bilstm_linear.csv'

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
best_accuracy = 0.0

In [14]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(ids_tensors[0])), 0)(ids_tensors[0])
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(tags_tensors[0])), 0)(tags_tensors[0])
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [15]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 4), tags_tensors.view(-1))
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [16]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [17]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [18]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = ABSA_BERT_Dropout_CNN_BiLSTM_Linear(BertModel.from_pretrained('bert-base-uncased'), bert_seq_len=SEQ_LEN, dropout=0.3, bilstm_in_features=256, conv_out_channels=SEQ_LEN, conv_kernel_size=3, no_out_labels=4, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/501, Loss: 1.4036377668380737
Epoch: 0/2, Batch: 50/501, Loss: 0.540870726108551
Epoch: 0/2, Batch: 100/501, Loss: 0.16217836737632751
Epoch: 0/2, Batch: 150/501, Loss: 0.07981859892606735
Epoch: 0/2, Batch: 200/501, Loss: 0.06743031740188599
Epoch: 0/2, Batch: 250/501, Loss: 0.04107315093278885
Epoch: 0/2, Batch: 300/501, Loss: 0.05272245779633522
Epoch: 0/2, Batch: 350/501, Loss: 0.032707009464502335
Epoch: 0/2, Batch: 400/501, Loss: 0.028365883976221085
Epoch: 0/2, Batch: 450/501, Loss: 0.02345585636794567
Epoch: 0/2, Batch: 500/501, Loss: 0.030286168679594994
Epoch: 1/2, Batch: 0/501, Loss: 0.041478853672742844
Epoch: 1/2, Batch: 50/501, Loss: 0.026294734328985214
Epoch: 1/2, Batch: 100/501, Loss: 0.04192529246211052
Epoch: 1/2, Batch: 150/501, Loss: 0.027593262493610382
Epoch: 1/2, Batch: 200/501, Loss: 0.0329744890332222
Epoch: 1/2, Batch: 250/501, Loss: 0.07231808453798294
Epoch: 1/2, Batch: 300/501, Loss: 0.019179590046405792
Epoch: 1/2, Batch: 350

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 2/10
Epoch: 0/2, Batch: 0/501, Loss: 1.4488288164138794
Epoch: 0/2, Batch: 50/501, Loss: 0.7619216442108154
Epoch: 0/2, Batch: 100/501, Loss: 0.18864081799983978
Epoch: 0/2, Batch: 150/501, Loss: 0.10109364986419678
Epoch: 0/2, Batch: 200/501, Loss: 0.04584861174225807
Epoch: 0/2, Batch: 250/501, Loss: 0.04234478622674942
Epoch: 0/2, Batch: 300/501, Loss: 0.03629087656736374
Epoch: 0/2, Batch: 350/501, Loss: 0.033240802586078644
Epoch: 0/2, Batch: 400/501, Loss: 0.0241562332957983
Epoch: 0/2, Batch: 450/501, Loss: 0.03447873890399933
Epoch: 0/2, Batch: 500/501, Loss: 0.031178640201687813
Epoch: 1/2, Batch: 0/501, Loss: 0.03549794480204582
Epoch: 1/2, Batch: 50/501, Loss: 0.029184814542531967
Epoch: 1/2, Batch: 100/501, Loss: 0.024362297728657722
Epoch: 1/2, Batch: 150/501, Loss: 0.017196688801050186
Epoch: 1/2, Batch: 200/501, Loss: 0.019786739721894264
Epoch: 1/2, Batch: 250/501, Loss: 0.02922510914504528
Epoch: 1/2, Batch: 300/501, Loss: 0.022457554936408997
Epoch: 1/2, Batch: 35

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 3/10
Epoch: 0/2, Batch: 0/501, Loss: 1.4013900756835938
Epoch: 0/2, Batch: 50/501, Loss: 0.6839701533317566
Epoch: 0/2, Batch: 100/501, Loss: 0.1788434535264969
Epoch: 0/2, Batch: 150/501, Loss: 0.07996433973312378
Epoch: 0/2, Batch: 200/501, Loss: 0.07365486025810242
Epoch: 0/2, Batch: 250/501, Loss: 0.05156903713941574
Epoch: 0/2, Batch: 300/501, Loss: 0.038105323910713196
Epoch: 0/2, Batch: 350/501, Loss: 0.09188424050807953
Epoch: 0/2, Batch: 400/501, Loss: 0.036511678248643875
Epoch: 0/2, Batch: 450/501, Loss: 0.029605833813548088
Epoch: 0/2, Batch: 500/501, Loss: 0.04514841362833977
Epoch: 1/2, Batch: 0/501, Loss: 0.03930458426475525
Epoch: 1/2, Batch: 50/501, Loss: 0.02343575470149517
Epoch: 1/2, Batch: 100/501, Loss: 0.02988218516111374
Epoch: 1/2, Batch: 150/501, Loss: 0.02104380913078785
Epoch: 1/2, Batch: 200/501, Loss: 0.014056169427931309
Epoch: 1/2, Batch: 250/501, Loss: 0.021474188193678856
Epoch: 1/2, Batch: 300/501, Loss: 0.030707253143191338
Epoch: 1/2, Batch: 350

...

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 9/10
Epoch: 0/2, Batch: 0/501, Loss: 1.3504616022109985
Epoch: 0/2, Batch: 50/501, Loss: 0.6222052574157715
Epoch: 0/2, Batch: 100/501, Loss: 0.16073337197303772
Epoch: 0/2, Batch: 150/501, Loss: 0.07605111598968506
Epoch: 0/2, Batch: 200/501, Loss: 0.03830961510539055
Epoch: 0/2, Batch: 250/501, Loss: 0.044893015176057816
Epoch: 0/2, Batch: 300/501, Loss: 0.050498880445957184
Epoch: 0/2, Batch: 350/501, Loss: 0.055002521723508835
Epoch: 0/2, Batch: 400/501, Loss: 0.035290997475385666
Epoch: 0/2, Batch: 450/501, Loss: 0.035957153886556625
Epoch: 0/2, Batch: 500/501, Loss: 0.021950624883174896
Epoch: 1/2, Batch: 0/501, Loss: 0.014182142913341522
Epoch: 1/2, Batch: 50/501, Loss: 0.02492327243089676
Epoch: 1/2, Batch: 100/501, Loss: 0.047729652374982834
Epoch: 1/2, Batch: 150/501, Loss: 0.02689414471387863
Epoch: 1/2, Batch: 200/501, Loss: 0.010870326310396194
Epoch: 1/2, Batch: 250/501, Loss: 0.03057052567601204
Epoch: 1/2, Batch: 300/501, Loss: 0.015139080584049225
Epoch: 1/2, Batch

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 10/10
Epoch: 0/2, Batch: 0/501, Loss: 1.3807799816131592
Epoch: 0/2, Batch: 50/501, Loss: 0.6890122294425964
Epoch: 0/2, Batch: 100/501, Loss: 0.2196011245250702
Epoch: 0/2, Batch: 150/501, Loss: 0.09113752096891403
Epoch: 0/2, Batch: 200/501, Loss: 0.05349485203623772
Epoch: 0/2, Batch: 250/501, Loss: 0.03679719939827919
Epoch: 0/2, Batch: 300/501, Loss: 0.04184240102767944
Epoch: 0/2, Batch: 350/501, Loss: 0.029359113425016403
Epoch: 0/2, Batch: 400/501, Loss: 0.05561598762869835
Epoch: 0/2, Batch: 450/501, Loss: 0.036041900515556335
Epoch: 0/2, Batch: 500/501, Loss: 0.02465047687292099
Epoch: 1/2, Batch: 0/501, Loss: 0.028066463768482208
Epoch: 1/2, Batch: 50/501, Loss: 0.02810288406908512
Epoch: 1/2, Batch: 100/501, Loss: 0.018674777820706367
Epoch: 1/2, Batch: 150/501, Loss: 0.025459708645939827
Epoch: 1/2, Batch: 200/501, Loss: 0.04705602303147316
Epoch: 1/2, Batch: 250/501, Loss: 0.01863742433488369
Epoch: 1/2, Batch: 300/501, Loss: 0.01275164820253849
Epoch: 1/2, Batch: 350

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.994711,0.994711,0.462972,0.994711,0.252889,0.994711,0.255044,485.788638
1,0.995094,0.995094,0.248784,0.995094,0.250000,0.995094,0.249391,482.152219
2,0.994844,0.994844,0.427284,0.994844,0.251253,0.994844,0.251842,482.395524
3,0.995191,0.995191,0.498801,0.995191,0.250555,0.995191,0.250506,483.724639
4,0.995137,0.995137,0.248784,0.995137,0.250000,0.995137,0.249391,482.391957
5,0.994844,0.994844,0.248711,0.994844,0.250000,0.994844,0.249354,485.672433
6,0.994926,0.994926,0.508809,0.994926,0.267298,0.994926,0.281769,480.104544
7,0.995277,0.995277,0.415488,0.995277,0.251098,0.995277,0.251594,485.306624
8,0.995004,0.995004,0.498761,0.995004,0.250495,0.995004,0.250366,482.151927
9,0.995379,0.995379,0.523880,0.995379,0.254150,0.995379,0.256702,486.071825


In [20]:
results.to_csv(STATS_OUTPUT)